Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [143]:
from random import random
from math import ceil
from functools import reduce
from collections import namedtuple, deque
from queue import PriorityQueue
import itertools

import numpy as np
from tqdm.auto import tqdm

In [144]:
State = namedtuple('State', ['taken', 'not_taken'])


def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )


def goal_check(state):
    return np.all(covered(state))

#print(State(set(range(NUM_SETS)),set()))

In [145]:
PROBLEM_SIZE = 15
NUM_SETS = 25
SETS = tuple(np.array([random() < 0.2 for _ in range(PROBLEM_SIZE)]) 
for _ in range(NUM_SETS))
assert goal_check(State(set(range(NUM_SETS)), set())), "Probelm not solvable"

## Depth First

In [146]:
frontier = deque()
state = State(set(), set(range(NUM_SETS)))
frontier.append(state)

counter = 0
current_state = frontier.pop()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.append(new_state)
        current_state = frontier.pop()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

0it [00:00, ?it/s]

Solved in 24 steps (24 tiles)


## Breadth First

In [147]:
# frontier = deque()
# state = State(set(), set(range(NUM_SETS)))
# frontier.append(state)

# counter = 0
# current_state = frontier.popleft()
# with tqdm(total=None) as pbar:
#     while not goal_check(current_state):
#         counter += 1
#         for action in current_state[1]:
#             new_state = State(
#                 current_state.taken ^ {action},
#                 current_state.not_taken ^ {action},
#             )
#             frontier.append(new_state)
#         current_state = frontier.popleft()
#         pbar.update(1)

# print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

## Greedy Best First

In [148]:
def f(state):
    missing_size = PROBLEM_SIZE - sum(covered(state))
    return missing_size

In [149]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((f(new_state), new_state))
        _, current_state = frontier.get()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

0it [00:00, ?it/s]

Solved in 5 steps (5 tiles)


## A*

In [150]:
def h(state):
    largest_set_size = max(sum(s) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(covered(state))
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h2(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    largest_set_size = max(sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(already_covered)
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h3(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken


def longest_string_size(set):
    max_length = max((sum(1 for _ in group) for key, group in itertools.groupby(set) if key), default=0)
    return max_length

def longest_string_in_takens(state):
    max_value = 1
    for index in state.taken:
        longest = longest_string_size(SETS[index])
        if longest > max_value:
            max_value = longest
    return max_value

def h4(state):
    '''first version of h4
    trying to build a heuristic that takes into account the length of longest string in selected sets.
    H(.) to be admissible should be <= PROBLEM_SIZE - missing_size'''
    #estimate = PROBLEM_SIZE / longest_string_in_takens(state)
    h_estimate = longest_string_in_takens(state) - 1
    
    #assert h_estimate >= sum(covered(state)), f"h(state): {h_estimate} seems h is not admissible"
    #distance = estimate * (PROBLEM_SIZE -1)
    return h_estimate


def f(state):
    missing_size = PROBLEM_SIZE - sum(covered(state))
    print (f"g: {missing_size} h: {h4(state)} problem_size - missing_size: {sum(covered(state))}")
    return missing_size + h4(state)

In [151]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((f(new_state), new_state))
        _, current_state = frontier.get()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

g: 15 h: 0 problem_size - missing_size: 0


0it [00:00, ?it/s]

g: 11 h: 0 problem_size - missing_size: 4
g: 11 h: 1 problem_size - missing_size: 4
g: 13 h: 1 problem_size - missing_size: 2
g: 10 h: 3 problem_size - missing_size: 5
g: 13 h: 0 problem_size - missing_size: 2
g: 11 h: 1 problem_size - missing_size: 4
g: 12 h: 0 problem_size - missing_size: 3
g: 11 h: 0 problem_size - missing_size: 4
g: 13 h: 0 problem_size - missing_size: 2
g: 13 h: 0 problem_size - missing_size: 2
g: 12 h: 0 problem_size - missing_size: 3
g: 10 h: 1 problem_size - missing_size: 5
g: 12 h: 1 problem_size - missing_size: 3
g: 12 h: 1 problem_size - missing_size: 3
g: 11 h: 0 problem_size - missing_size: 4
g: 11 h: 1 problem_size - missing_size: 4
g: 14 h: 0 problem_size - missing_size: 1
g: 10 h: 0 problem_size - missing_size: 5
g: 12 h: 1 problem_size - missing_size: 3
g: 12 h: 0 problem_size - missing_size: 3
g: 14 h: 0 problem_size - missing_size: 1
g: 12 h: 0 problem_size - missing_size: 3
g: 13 h: 0 problem_size - missing_size: 2
g: 13 h: 0 problem_size - missing_